# 0. Загрузка библиотек

In [175]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn import model_selection, linear_model, metrics
from sklearn.preprocessing import StandardScaler

plt.rcParams.update({'figure.max_open_warning': 0})
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import joblib
import pickle
import tensorflow as tf

In [176]:
import tensorflow as tf
print("GPU Available:", tf.test.is_gpu_available())

GPU Available: False


In [177]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4640808398048396397
xla_global_id: -1
]


# 1. Загрузка данных

In [178]:
# загрузка тестовой выборки
with open("/kaggle/input/ml-cource-cifrum-anomaly-public/test.pkl", "rb") as f:
    list_of_df = pickle.load(f)


In [179]:
len(pd.concat(list_of_df))

37401

# 2. Подготовка данных

In [180]:
for ind in range(len(list_of_df)):
    # признаки, имеющие физический смысл
    # мощность
    list_of_df[ind]["Power"] = list_of_df[ind]["Current"] * list_of_df[ind]["Voltage"] 
    # отношение расхода к мощности
    list_of_df[ind]["Power_flow_rate"] = list_of_df[ind]["Volume Flow RateRMS"] / list_of_df[ind]["Power"]
    # разница температур (если признаки скоррелированы и с одним из них что-то происходит, то покажет наличие аномалии)
    list_of_df[ind]["Temperature_diff"] = list_of_df[ind]['Temperature'] - list_of_df[ind]['Thermocouple'] 
    # разница акселлерометров (если признаки скоррелированы и с одним из них что-то происходит, то покажет наличие аномалии)
    list_of_df[ind]["Accel_diff"] = list_of_df[ind]['Accelerometer1RMS'] - list_of_df[ind]['Accelerometer2RMS']
    
    # сглаживания
    list_of_df[ind]["Volume Flow RateRMS_10mean"] = list_of_df[ind]["Volume Flow RateRMS"].rolling(window = 10, min_periods=0).mean()
   

# 3. Инициализация и обучение модели

In [181]:
from cnn_ae import Conv_AE

In [182]:
EPOCHS = 6
BATCH_SIZE = 32
VAL_SPLIT = 0.1
N_STEPS = 60
Q = 0.999

In [183]:
# функция для генерации выборок для обучения
def create_sequences(values, time_steps=N_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

In [184]:
%%time
# инференс
predicted_outlier, predicted_cp = [], []
with tf.device("/device:GPU:0"):
    for df in list_of_df:
        X_train = df[:400]

        # инициализация и обучение нормализатора данных
        StSc = StandardScaler()
        StSc.fit(X_train)

        # масштабирование данных и генерация выборок для обучения
        X = create_sequences(StSc.transform(X_train), N_STEPS)

        # инициализация модели
        model = Conv_AE()
        # обучение модели
        model.fit(X)

        # прогноз на обучающей выборке и выбор порога
        residuals = pd.Series(np.sum(np.mean(np.abs(X - model.predict(X)), axis=1), axis=1))
        UCL = residuals.quantile(Q) * 1.16

        # прогноз на всей выборке и поиск аномалий по порогу
        X = create_sequences(StSc.transform(df), N_STEPS)
        cnn_residuals = pd.Series(np.sum(np.mean(np.abs(X - model.predict(X)), axis=1), axis=1))
        
        anomalous_data = cnn_residuals > UCL
        anomalous_data_indices = []
        for data_idx in range(N_STEPS - 1, len(X) - N_STEPS + 1):
            if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
                anomalous_data_indices.append(data_idx)

        prediction = pd.Series(data=0, index=df.index)
        prediction.iloc[anomalous_data_indices] = 1

        # сохранение результатов предсказания аномалий
        predicted_outlier.append(prediction)

        # сохранение предсказания точек изменения состояния
        prediction_cp = abs(prediction.diff())
        prediction_cp[0] = prediction[0]
        predicted_cp.append(prediction_cp)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
34/34 ━━━━━━

# 4. Формирование и сохранение предсказания моделью

In [185]:
# формирование вектора предсказаний по всем экспериментам
pred = pd.concat(predicted_outlier)

# формирование массива для загрузки на платформу
y_pred=pd.Series(pred, name = "anomaly")
y_pred.index = np.arange(0, y_pred.shape[0])

y_pred = y_pred.reset_index()
y_pred.columns = ["ID", "anomaly"]
y_pred.to_csv("predict11.csv", index = False)

In [186]:
# отображение созданного массива с предсказанием
y_pred

,ID,anomaly
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
37396,37396,0
37397,37397,0
37398,37398,0
37399,37399,0


In [187]:
y_pred["anomaly"].value_counts()

anomaly
0    24521
1    12880
Name: count, dtype: int64

In [188]:
import joblib
joblib.dump(model, "model11.pkl")

['model11.pkl']